<a href="https://colab.research.google.com/github/Muhammad-ali-aren/ResNet-Architecture-ResNet18/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import torch
import torch.nn as nn
import torchsummary

In [52]:
class ResNet_layer(nn.Module):
    def __init__(self,input_channels,output_channels,stride=1):
        super().__init__()
        self.conv3x_1 = nn.Conv2d(in_channels=input_channels,out_channels=output_channels,kernel_size=3,stride=stride,padding=1)
        self.batchn1 = nn.BatchNorm2d(output_channels)
        self.conv3x_2 = nn.Conv2d(in_channels=output_channels,out_channels=output_channels,kernel_size=3,padding=1)
        self.batchn2 = nn.BatchNorm2d(output_channels)
        if input_channels != output_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels=input_channels,out_channels=output_channels,kernel_size=1,stride=stride),
                nn.BatchNorm2d(output_channels)
            )
        else:
            self.shortcut = nn.Identity()
    def forward(self,input):
        out_shortcut = self.shortcut(input)
        out1 = torch.relu(self.batchn1(self.conv3x_1(input)))
        out2 = self.batchn2(self.conv3x_2(out1))
        out = torch.relu(out2 + out_shortcut)
        return out

In [70]:
class ResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.conv2_R = ResNet_layer(input_channels=64,output_channels=64)
        self.conv3_R = ResNet_layer(input_channels=64,output_channels=128,stride=2)
        self.conv4_R = ResNet_layer(input_channels=128,output_channels=256,stride=2)
        self.conv5_R = ResNet_layer(input_channels=256,output_channels=512,stride=2)
        self.avg_pool = nn.AvgPool2d(kernel_size=7)
        self.fc1 = nn.Linear(1*1*512, 4096)
        self.fc2 = nn.Linear(4096,1000)
    def forward(self,input):
        x = self.max_pool1(self.conv1(input))
        x = self.conv3_R(self.conv2_R(x))
        x = self.conv5_R(self.conv4_R(x))
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        out = self.fc2(self.fc1(x))
        return out

In [77]:
model = ResNet18()
torchsummary.summary(model,input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
         MaxPool2d-2           [-1, 64, 56, 56]               0
          Identity-3           [-1, 64, 56, 56]               0
            Conv2d-4           [-1, 64, 56, 56]          36,928
       BatchNorm2d-5           [-1, 64, 56, 56]             128
            Conv2d-6           [-1, 64, 56, 56]          36,928
       BatchNorm2d-7           [-1, 64, 56, 56]             128
      ResNet_layer-8           [-1, 64, 56, 56]               0
            Conv2d-9          [-1, 128, 28, 28]           8,320
      BatchNorm2d-10          [-1, 128, 28, 28]             256
           Conv2d-11          [-1, 128, 28, 28]          73,856
      BatchNorm2d-12          [-1, 128, 28, 28]             256
           Conv2d-13          [-1, 128, 28, 28]         147,584
      BatchNorm2d-14          [-1, 128,

In [71]:
model = ResNet18()
model(torch.randn(1,3,224,224)).shape

torch.Size([1, 1000])

In [ ]:
!pip install ptflops

In [79]:
from ptflops import get_model_complexity_info

flops, params = get_model_complexity_info(model, (3, 224, 224), as_strings=True, print_per_layer_stat=True)
print(f"FLOPs: {flops}")
print(f"Params: {params}")

ResNet18(
  11.11 M, 100.000% Params, 899.33 MMac, 99.908% MACs, 
  (conv1): Conv2d(9.47 k, 0.085% Params, 118.82 MMac, 13.200% MACs, 3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (max_pool1): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.089% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2_R): ResNet_layer(
    74.11 k, 0.667% Params, 232.42 MMac, 25.819% MACs, 
    (conv3x_1): Conv2d(36.93 k, 0.332% Params, 115.81 MMac, 12.865% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchn1): BatchNorm2d(128, 0.001% Params, 401.41 KMac, 0.045% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3x_2): Conv2d(36.93 k, 0.332% Params, 115.81 MMac, 12.865% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchn2): BatchNorm2d(128, 0.001% Params, 401.41 KMac, 0.045% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Identity(0, 0.000% Params, 